# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [6]:
import numpy as np
import time
from os.path import exists
from math import sqrt

Будем использовать numpy, т.к. он гораздо быстрее

In [7]:
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

def euclidean_distance(point1, point2):
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)
    
def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], 
                                  instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [9]:
def solve_tsp_nearest_neighbour(instance):
    start = 0
    INF = 10**9 + 7
    n = len(instance)
    ans = [start]
    used = [False] * n
    used[start] = True
    for i in range(n - 1):
        index = -1
        min_dist = INF
        for j in range(n):
            if not used[j]:
                d = euclidean_distance(instance[ans[i]], instance[j])
                if d < min_dist:
                    min_dist = d
                    index = j
        ans.append(index)
        used[index] = True
    return ans

In [26]:
def solve_tsp_nearest_insertion(instance):
    def weight(i, j):
        return euclidean_distance(instance[i], instance[j])
    n = len(instance)
    ans = [0]
    used = [False] * n
    used[0] = True
    dst = [weight(0, i) for i in range(1, n)]
    ans.append(dst.index(min(dst)) + 1)
    used[ans[1]] = True
    a = list(range(1, n))
    not_in_ans = a[:ans[1] - 1] + a[ans[1]:]
    for i in range(n - 2):
        k = len(ans)
        m = n - len(ans)
        X = np.array([list(instance[i]) for i in ans]).T
        Y = np.array([list(instance[i]) for i in not_in_ans]).T
        X1 = np.tile(X, m).reshape((2, m, k))
        X1 = np.array([x.T for x in X1])
        Y1 = np.tile(Y, k).reshape((2, k, m))
        D = np.sqrt((X1[0] - Y1[0]) ** 2 + (X1[1] - Y1[1]) ** 2)
        dist_v_u1_v_u2 = D + np.roll(D, 1, axis=0)
        neigh_dist_in_ans = np.array([weight(ans[i - 1], ans[i]) for i in range(k)])
        M = dist_v_u1_v_u2.T - neigh_dist_in_ans
        p = np.argmin(dist_v_u1_v_u2.T - neigh_dist_in_ans)
        min_delta = np.min(dist_v_u1_v_u2.T - neigh_dist_in_ans)
        v = not_in_ans[p // k]
        u1 = ans[p % k - 1]
        u2 = ans[p % k]
        if i == 0:
            for j in range(k):
                for t in range(m):
                    cur_d = weight(ans[j], not_in_ans[t]) + weight(ans[j - 1],
                                                                   not_in_ans[t]) - weight(ans[j - 1], ans[j])
        ans = ans[:p % k] + [v] + ans[p % k:]
        not_in_ans = not_in_ans[:p // k] + not_in_ans[p // k + 1:]
    return ans

In [21]:
def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print('''done in {:.2} seconds with tour length {} using NN and in {:.2} seconds with tour 
                 length {} using NI'''.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [24]:
run_all()

Solving instance d198.tsp…done in 0.045 seconds with tour length 18620 using NN and in 0.17 seconds with tour 
                 length 17631 using NI
Solving instance d493.tsp…done in 0.17 seconds with tour length 43646 using NN and in 1.4 seconds with tour 
                 length 39982 using NI
Solving instance d657.tsp…done in 0.29 seconds with tour length 62176 using NN and in 3.0 seconds with tour 
                 length 57906 using NI
Solving instance d2103.tsp…done in 3.0 seconds with tour length 87468 using NN and in 1.1e+02 seconds with tour 
                 length 87530 using NI
Solving instance pr107.tsp…done in 0.0081 seconds with tour length 46678 using NN and in 0.047 seconds with tour 
                 length 51667 using NI
Solving instance pr152.tsp…done in 0.015 seconds with tour length 85702 using NN and in 0.087 seconds with tour 
                 length 88530 using NI
Solving instance pr439.tsp…done in 0.13 seconds with tour length 131282 using NN and in 0.95 seco

** Выводы **

NI дает результаты чуть лучше, но не всегда, видно, что иногда NN лучше. Также NI работает на порядок дольше, что очевидно из ассимптотики.